In [42]:
! pip install rdkit

In [43]:
import pandas as pd
import numpy as np

from rdkit import Chem

In [44]:
df_train = pd.read_csv("train.csv")

In [45]:
df_train["smiles_to_delete"] = df_train["smiles"]

1 - We create one column per element

In [46]:
#One column per element 
#No He, Li, Be, Ne, Ar, K, Sc, V, Mn, Fe, Ni, Cu, Kr, Rb, Sr, Y, Zr, Nb, Mo, Tc, Ru, Rh, Pd and after in the dataset
df_train["Number of H"] = 0
df_train["Number of B"] = 0
df_train["Number of C"] = 0
df_train["Number of N"] = 0
df_train["Number of O"] = 0
df_train["Number of F"] = 0
df_train["Number of Si"] = 0
df_train["Number of P"] = 0
df_train["Number of S"] = 0
df_train["Number of Cl"] = 0
df_train["Number of Br"] = 0
df_train["Number of I"] = 0

#In the dataset but very few amount (so drop these lines):
#Na (2 lines) Mg (1 line) Al (1 line) Ca (1 line) Ti (1 line) Cr (1 line) [Co] (1 line) Zn (1 line) Ge (1 line) As (4 lines)
#Se (7 lines) Ag (1 line) [Hg] (2 lines)

2 - We remove in "smiles_to_delete", the elements that appears less than 10 times in the dataset:

In [47]:
element_to_remove = ["Na", "Mg", "Al", "Ca", "Ti", "Cr", "[Co]", "Zn", "Ge", "As", "Se", "Ag", "[Hg]"]

# Liste pour stocker les indices des lignes à supprimer
indices_to_remove = []

for index, row in df_train.iterrows():
    for element in element_to_remove:
        if element in row["smiles_to_delete"]:
            indices_to_remove.append(index)  # Ajoutez l'indice de la ligne à supprimer

# Supprimez les lignes correspondantes
df_train.drop(indices_to_remove, inplace=True)

# Réinitialisez les index du DataFrame
df_train.reset_index(drop=True, inplace=True)

3 - We remove the special characters

In [48]:
char_to_remove = ["(", ")", "=", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "@", "[", "]", "-", "+", "H", "/", "#", ".", "\\", "%", " "] 
                  #We remove H car it is not suppose to be in smile

for index, row in df_train.iterrows():
    cleaned_smiles = ''.join([char for char in row["smiles_to_delete"] if char not in char_to_remove])
    df_train.loc[index, "smiles_to_delete"] = cleaned_smiles
                  

4- We remove the important elements from smiles_to_delete and add the count in the column

In [49]:
element_to_count = ["Br", "B", "Cl", "C", "N", "O", "F", "Si", "P", "S", "I"]  # L'ordre est important !!!

for index, row in df_train.iterrows():
    for element in element_to_count:
        count = row["smiles_to_delete"].count(element.lower()) + row["smiles_to_delete"].count(element)
        df_train.loc[index, "Number of " + element] = count  # Ajouter la colonne pour le nombre d'occurrences
        row["smiles_to_delete"] = row["smiles_to_delete"].replace(element.lower(), "").replace(element, "")  # Supprimer l'élément de la chaîne (en ignorant la casse)
    # Mettre à jour la colonne "smiles_to_delete" dans le DataFrame
    df_train.loc[index, "smiles_to_delete"] = row["smiles_to_delete"]

5- Let's count the number and types of bond (Not H)

In [50]:
from rdkit import Chem

# Supposons que vous ayez déjà un DataFrame df_train avec une colonne "smiles"

# Fonction pour compter les liaisons par type
def count_bond_types(smiles):
    mol = Chem.MolFromSmiles(smiles)
    bond_counts = {}
    
    if mol:
        for bond in mol.GetBonds():
            begin_atom = bond.GetBeginAtom().GetSymbol()
            end_atom = bond.GetEndAtom().GetSymbol()
            bond_type = bond.GetBondTypeAsDouble()  # Obtenir le type de liaison
            
            bond_key = f"{begin_atom}-{end_atom} ({bond_type})"
            if bond_key in bond_counts:
                bond_counts[bond_key] += 1
            else:
                bond_counts[bond_key] = 1
    
    return bond_counts

# Appliquez la fonction count_bond_types à chaque SMILES et créez une nouvelle colonne 'Bond Counts'
df_train['Bond Counts'] = df_train['smiles'].apply(count_bond_types)

# Affichez le DataFrame résultant
print(df_train)

[19:43:47] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13
[19:43:47] Explicit valence for atom # 8 N, 4, is greater than permitted
[19:43:47] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 10 11 12
[19:43:47] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 17 18
[19:43:47] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[19:43:47] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
[19:43:47] Explicit valence for atom # 20 C, 5, is greater than permitted
[19:43:47] Explicit valence for atom # 5 N, 4, is greater than permitted
[19:43:47] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[19:43:47] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 6
[19:43:47] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15 16 17
[19:43:47] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9
[19:43:47] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[19:43:47] Explicit valence for atom # 6 N, 5, is greater than permitted
[19:43:47] Can't kekulize mol.  U

[19:43:49] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 17 18
[19:43:49] Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20 21 22 23 24
[19:43:49] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11
[19:43:49] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 8 9 10
[19:43:49] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11
[19:43:49] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
[19:43:50] Can't kekulize mol.  Unkekulized atoms: 26 27 28 29 30 33 34 35 36
[19:43:50] Can't kekulize mol.  Unkekulized atoms: 18 19 20 21 22
[19:43:50] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11 12 13 14 15
[19:43:50] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15 16 17
[19:43:50] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13
[19:43:50] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 11 12 14 15
[19:43:50] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13
[19:43:50] Can't kekulize mol.  Unkekulized atoms: 0 1 

                                                    name  \
0                       3-i-pr-5-mephenyl-n-me carbamate   
1                                  b-2-furylacrylic acid   
2                                            cyclacillin   
3                         2,6-dihydroxy-4-methylpyridine   
4                                      dimethylnitramine   
...                                                  ...   
19906  diethyl 3,3-bis(4-hydroxy-3,5-dimethylphenyl)p...   
19907                                    diethyl azelate   
19908                             4-methyl-3-nitrophenol   
19909                        (s)-3-(boc-amino)piperidine   
19910                                 acetylpyruvic acid   

                                                  smiles    mpC  \
0                             O=C(Oc1cc(C)cc(c1)C(C)C)NC   87.0   
1                                       O=C(O)C=Cc1occc1  133.0   
2           OC(=O)C2N3C(=O)C(NC(=O)C1(N)CCCCC1)C3SC2(C)C  182.5   
3          

[19:43:50] Can't kekulize mol.  Unkekulized atoms: 12 13 15 16 17 18 19 20 21
[19:43:50] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9


6-Let's count the number of bond involving H

In [51]:
from rdkit import Chem
import pandas as pd

# Supposons que vous avez déjà votre DataFrame df_train avec une colonne 'smiles'

# Définissez les motifs SMARTS pour les groupes fonctionnels contenant H
ch3_pattern = Chem.MolFromSmarts('[CH3]')
ch2_pattern = Chem.MolFromSmarts('[CH2]')
oh_pattern = Chem.MolFromSmarts('[OH]')
ch_pattern = Chem.MolFromSmarts('[CH]')
ch_pattern_ar = Chem.MolFromSmarts('[cH]')
bh_pattern = Chem.MolFromSmarts('[BH]')
bh_pattern_ar = Chem.MolFromSmarts('[bH]')
bh2_pattern = Chem.MolFromSmarts('[BH2]')
bh3_pattern = Chem.MolFromSmarts('[BH3]')
nh_pattern = Chem.MolFromSmarts('[NH]')
nh_pattern_ar = Chem.MolFromSmarts('[nH]')
nh2_pattern = Chem.MolFromSmarts('[NH2]')
nh3_pattern = Chem.MolFromSmarts('[NH3]')
sih_pattern = Chem.MolFromSmarts('[SiH]')
sih2_pattern = Chem.MolFromSmarts('[SiH2]')
sih3_pattern = Chem.MolFromSmarts('[SiH3]')
sih4_pattern = Chem.MolFromSmarts('[SiH4]')
ph_pattern = Chem.MolFromSmarts('[PH]')
ph_pattern_ar = Chem.MolFromSmarts('[pH]')
ph2_pattern = Chem.MolFromSmarts('[PH2]')
ph3_pattern = Chem.MolFromSmarts('[PH3]')
sh_pattern = Chem.MolFromSmarts('[SH]')
sh_pattern_ar = Chem.MolFromSmarts('[sH]')

# Fonction pour compter les groupes fonctionnels contenant H
def count_h_containing_groups(smiles):
    mol = Chem.MolFromSmiles(smiles)
    
    if mol:
        group_counts = {
            'CH3': len(mol.GetSubstructMatches(ch3_pattern)),
            'CH2': len(mol.GetSubstructMatches(ch2_pattern)),
            'OH': len(mol.GetSubstructMatches(oh_pattern)),
            'CH': len(mol.GetSubstructMatches(ch_pattern)) + len(mol.GetSubstructMatches(ch_pattern_ar)),
            'BH': len(mol.GetSubstructMatches(bh_pattern)) + len(mol.GetSubstructMatches(bh_pattern_ar)),
            'Bh2': len(mol.GetSubstructMatches(bh2_pattern)),
            'Bh3': len(mol.GetSubstructMatches(bh3_pattern)),
            'NH': len(mol.GetSubstructMatches(nh_pattern)) + len(mol.GetSubstructMatches(nh_pattern_ar)),
            'NH2': len(mol.GetSubstructMatches(nh2_pattern)),
            'NH3': len(mol.GetSubstructMatches(nh3_pattern)),
            'SiH': len(mol.GetSubstructMatches(sih_pattern)),
            'SiH2': len(mol.GetSubstructMatches(sih2_pattern)),
            'SiH3': len(mol.GetSubstructMatches(sih3_pattern)),
            'SiH4': len(mol.GetSubstructMatches(sih4_pattern)),
            'PH': len(mol.GetSubstructMatches(ph_pattern)) + len(mol.GetSubstructMatches(ph_pattern_ar)),
            'PH2': len(mol.GetSubstructMatches(ph2_pattern)),
            'PH3': len(mol.GetSubstructMatches(ph3_pattern)),
            'SH': len(mol.GetSubstructMatches(sh_pattern))+ len(mol.GetSubstructMatches(sh_pattern_ar))
        }
        
        return group_counts
    
    else:
        return None

# Appliquez la fonction count_h_containing_groups à chaque SMILES et créez de nouvelles colonnes pour chaque groupe fonctionnel
df_train['Functional Groups with H'] = df_train['smiles'].apply(count_h_containing_groups)

# Affichez le DataFrame résultant
print(df_train)


[19:43:51] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13
[19:43:51] Explicit valence for atom # 8 N, 4, is greater than permitted
[19:43:51] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 10 11 12
[19:43:51] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 17 18
[19:43:51] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[19:43:51] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
[19:43:51] Explicit valence for atom # 20 C, 5, is greater than permitted
[19:43:51] Explicit valence for atom # 5 N, 4, is greater than permitted
[19:43:51] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[19:43:51] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 6
[19:43:51] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15 16 17
[19:43:51] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9
[19:43:51] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[19:43:51] Explicit valence for atom # 6 N, 5, is greater than permitted
[19:43:51] Can't kekulize mol.  U

[19:43:53] Can't kekulize mol.  Unkekulized atoms: 33 34 35 36 37
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 14
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 11 12 13 15 16 17 18 19 21 22 23
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 10 11 12
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16 17 18 19 20
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 12
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 20 21 22
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 6 7 8 9 11 12 13 15 16 17 18
[19:43:53] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13

                                                    name  \
0                       3-i-pr-5-mephenyl-n-me carbamate   
1                                  b-2-furylacrylic acid   
2                                            cyclacillin   
3                         2,6-dihydroxy-4-methylpyridine   
4                                      dimethylnitramine   
...                                                  ...   
19906  diethyl 3,3-bis(4-hydroxy-3,5-dimethylphenyl)p...   
19907                                    diethyl azelate   
19908                             4-methyl-3-nitrophenol   
19909                        (s)-3-(boc-amino)piperidine   
19910                                 acetylpyruvic acid   

                                                  smiles    mpC  \
0                             O=C(Oc1cc(C)cc(c1)C(C)C)NC   87.0   
1                                       O=C(O)C=Cc1occc1  133.0   
2           OC(=O)C2N3C(=O)C(NC(=O)C1(N)CCCCC1)C3SC2(C)C  182.5   
3          

[19:43:54] Can't kekulize mol.  Unkekulized atoms: 12 13 15 16 17 18 19 20 21
[19:43:54] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9


Quick check

In [52]:
boron_containing_smiles = df_train[df_train['smiles'].str.contains('B')]

# Affichez les lignes correspondantes
print(boron_containing_smiles)

                                                    name  \
9                         3-methyl-2,4,6-tribromoaniline   
19                        benzene, 1,2-bis(bromomethyl)-   
24      2-amino-3,5-dibromo-n-cyclohexyl-n-methylbenzene   
38                                   4-bromo-benzylamine   
87     (2e)-3-(5-bromo-8-nitronaphthalen-1-yl)prop-2-...   
...                                                  ...   
19850             2-(methoxycarbonyl)benzeneboronic acid   
19853                                2,5-dibromothiazole   
19854                 3-(bromomethyl)benzeneboronic acid   
19860  (1s,8s)-13-bromo-3,6-dimethyl-1-(2-methylbenzy...   
19877                    2-amino-3,5-dibromobenzoic acid   

                                                  smiles    mpC  \
9                                 Cc1c(cc(c(c1Br)N)Br)Br  101.0   
19                                        BrCc1ccccc1CBr   95.0   
24                        CN(Cc1cc(Br)cc(Br)c1N)C2CCCCC2  238.0   
38         

In [53]:
print(df_train.at[19854, "Functional Groups with H"])
print(df_train.at[19854, "Bond Counts"])
print(df_train.at[19854, "smiles"])

{'CH3': 0, 'CH2': 1, 'OH': 2, 'CH': 4, 'BH': 0, 'Bh2': 0, 'Bh3': 0, 'NH': 0, 'NH2': 0, 'NH3': 0, 'SiH': 0, 'SiH2': 0, 'SiH3': 0, 'SiH4': 0, 'PH': 0, 'PH2': 0, 'PH3': 0, 'SH': 0}
{'B-C (1.0)': 1, 'C-C (1.5)': 6, 'C-C (1.0)': 1, 'C-Br (1.0)': 1, 'B-O (1.0)': 2}
B(c1cccc(c1)CBr)(O)O


In [54]:
df_train.drop(columns="smiles_to_delete", inplace=True)
df_train

,name,smiles,mpC,Number of H,Number of B,Number of C,Number of N,Number of O,Number of F,Number of Si,Number of P,Number of S,Number of Cl,Number of Br,Number of I,Bond Counts,Functional Groups with H
0,3-i-pr-5-mephenyl-n-me carbamate,O=C(Oc1cc(C)cc(c1)C(C)C)NC,87.0,0,0,12,1,2,0,0,0,0,0,0,0,"{'O-C (2.0)': 1, 'C-O (1.0)': 1, 'O-C (1.0)': ...","{'CH3': 4, 'CH2': 0, 'OH': 0, 'CH': 4, 'BH': 0..."
1,b-2-furylacrylic acid,O=C(O)C=Cc1occc1,133.0,0,0,7,0,3,0,0,0,0,0,0,0,"{'O-C (2.0)': 1, 'C-O (1.0)': 1, 'C-C (1.0)': ...","{'CH3': 0, 'CH2': 0, 'OH': 1, 'CH': 5, 'BH': 0..."
2,cyclacillin,OC(=O)C2N3C(=O)C(NC(=O)C1(N)CCCCC1)C3SC2(C)C,182.5,0,0,15,3,4,0,0,0,1,0,0,0,"{'O-C (1.0)': 1, 'C-O (2.0)': 3, 'C-C (1.0)': ...","{'CH3': 2, 'CH2': 5, 'OH': 1, 'CH': 3, 'BH': 0..."
3,"2,6-dihydroxy-4-methylpyridine",Cc1cc(nc(c1)O)O,197.0,0,0,6,1,2,0,0,0,0,0,0,0,"{'C-C (1.0)': 1, 'C-C (1.5)': 4, 'C-N (1.5)': ...","{'CH3': 1, 'CH2': 0, 'OH': 2, 'CH': 2, 'BH': 0..."
4,dimethylnitramine,CN(C)[N+]([O-])=O,58.0,0,0,2,2,2,0,0,0,0,0,0,0,"{'C-N (1.0)': 1, 'N-C (1.0)': 1, 'N-N (1.0)': ...","{'CH3': 2, 'CH2': 0, 'OH': 0, 'CH': 0, 'BH': 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,"diethyl 3,3-bis(4-hydroxy-3,5-dimethylphenyl)p...",O=C(OCC)CC(CC(=O)OCC)(c1cc(C)c(O)c(C)c1)c1cc(C...,178.0,0,0,25,0,6,0,0,0,0,0,0,0,"{'O-C (2.0)': 1, 'C-O (1.0)': 4, 'O-C (1.0)': ...","{'CH3': 6, 'CH2': 4, 'OH': 2, 'CH': 4, 'BH': 0..."
19907,diethyl azelate,O=C(CCCCCCCC(=O)OCC)OCC,-18.5,0,0,13,0,4,0,0,0,0,0,0,0,"{'O-C (2.0)': 1, 'C-C (1.0)': 10, 'C-O (2.0)':...","{'CH3': 2, 'CH2': 9, 'OH': 0, 'CH': 0, 'BH': 0..."
19908,4-methyl-3-nitrophenol,Cc1ccc(cc1[N+](=O)[O-])O,79.0,0,0,7,1,3,0,0,0,0,0,0,0,"{'C-C (1.0)': 1, 'C-C (1.5)': 6, 'C-N (1.0)': ...","{'CH3': 1, 'CH2': 0, 'OH': 1, 'CH': 3, 'BH': 0..."
19909,(s)-3-(boc-amino)piperidine,CC(C)(C)OC(=O)N[C@H]1CCCNC1,125.0,0,0,10,2,2,0,0,0,0,0,0,0,"{'C-C (1.0)': 7, 'C-O (1.0)': 1, 'O-C (1.0)': ...","{'CH3': 3, 'CH2': 4, 'OH': 0, 'CH': 1, 'BH': 0..."


In [55]:
functional_groups_counts = {
    'CH3': 0,
    'CH2': 1,
    'OH': 2,
    'CH': 4,
    'BH': 0,
    'Bh2': 0,
    'Bh3': 0,
    'NH': 0,
    'NH2': 0,
    'NH3': 0,
    'SiH': 0,
    'SiH2': 0,
    'SiH3': 0,
    'SiH4': 0,
    'PH': 0,
    'PH2': 0,
    'PH3': 0,
    'SH': 0
}

# Utilisez apply pour créer une colonne pour chaque clé
for group_name, group_count in functional_groups_counts.items():
    df_train[group_name] = df_train['Functional Groups with H'].apply(lambda x: x.get(group_name, 0) if x is not None else 0)


In [56]:
df_train

,name,smiles,mpC,Number of H,Number of B,Number of C,Number of N,Number of O,Number of F,Number of Si,...,NH2,NH3,SiH,SiH2,SiH3,SiH4,PH,PH2,PH3,SH
0,3-i-pr-5-mephenyl-n-me carbamate,O=C(Oc1cc(C)cc(c1)C(C)C)NC,87.0,0,0,12,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,b-2-furylacrylic acid,O=C(O)C=Cc1occc1,133.0,0,0,7,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,cyclacillin,OC(=O)C2N3C(=O)C(NC(=O)C1(N)CCCCC1)C3SC2(C)C,182.5,0,0,15,3,4,0,0,...,1,0,0,0,0,0,0,0,0,0
3,"2,6-dihydroxy-4-methylpyridine",Cc1cc(nc(c1)O)O,197.0,0,0,6,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,dimethylnitramine,CN(C)[N+]([O-])=O,58.0,0,0,2,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,"diethyl 3,3-bis(4-hydroxy-3,5-dimethylphenyl)p...",O=C(OCC)CC(CC(=O)OCC)(c1cc(C)c(O)c(C)c1)c1cc(C...,178.0,0,0,25,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
19907,diethyl azelate,O=C(CCCCCCCC(=O)OCC)OCC,-18.5,0,0,13,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
19908,4-methyl-3-nitrophenol,Cc1ccc(cc1[N+](=O)[O-])O,79.0,0,0,7,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
19909,(s)-3-(boc-amino)piperidine,CC(C)(C)OC(=O)N[C@H]1CCCNC1,125.0,0,0,10,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
#1 -Il y a un moyen d'avoir les groupes fonctionnels avec rdkit (taper functional grope rdkit)

In [58]:
#2 -Maintenant que j'ai tout les groupes avec H, faire une soustraction pour calculer les atomes sans H

In [59]:
#3 -Maintenant que j'ai tout, calculer le nombre d'insaturation et commencer les modeles (on pourra toujours ameliorer plus tard)

In [60]:
#4-Cleaner le notebooks et bien regarder si c'est correct (utiliser le site : https://www.novoprolabs.com/tools/smiles2pdb)

In [61]:
#5- Commencer les modeles

Feature engineering is not finish, but lets see some models:

***Model 1***

In [62]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

X = df_train.drop(columns=["name", "smiles", "mpC", "Functional Groups with H", "Bond Counts"])
y = df_train["mpC"]

model = LinearRegression()
model.fit(X, y)

scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')  # Métrique de performance : Erreur quadratique moyenne négative

# Le score sera négatif, donc prenez l'opposé pour obtenir l'erreur quadratique moyenne positive
rmse_scores = (-scores) ** 0.5

# Calculez la moyenne des scores RMSE
mean_rmse_score = rmse_scores.mean()
print("Moyenne des scores RMSE :", mean_rmse_score)


Moyenne des scores RMSE : 65.45641807139722


In [63]:
model.score(X, y)

0.5014628534591004

***Model 2***

In [64]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# Créez un modèle de régression polynomiale de degré 2
degree = 2
model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Entraînez le modèle
model.fit(X, y)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [65]:
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')  # Métrique de performance : Erreur quadratique moyenne négative

# Le score sera négatif, donc prenez l'opposé pour obtenir l'erreur quadratique moyenne positive
rmse_scores = (-scores) ** 0.5

# Calculez la moyenne des scores RMSE
mean_rmse_score = rmse_scores.mean()
print("Moyenne des scores RMSE :", mean_rmse_score)

Moyenne des scores RMSE : 613591899.9077444


***Model 3***

In [66]:
from sklearn.ensemble import RandomForestRegressor

# Créez un modèle de forêt aléatoire
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Entraînez le modèle
model.fit(X, y)

RandomForestRegressor(random_state=0)

In [67]:
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')  # Métrique de performance : Erreur quadratique moyenne négative

# Le score sera négatif, donc prenez l'opposé pour obtenir l'erreur quadratique moyenne positive
rmse_scores = (-scores) ** 0.5

# Calculez la moyenne des scores RMSE
mean_rmse_score = rmse_scores.mean()
print("Moyenne des scores RMSE :", mean_rmse_score)

Moyenne des scores RMSE : 48.51759681715769


***Model 4***

In [68]:
from sklearn.neural_network import MLPRegressor

# Créez un modèle de réseau de neurones
model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000)

# Entraînez le modèle
model.fit(X, y)

MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000)

In [69]:
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')  # Métrique de performance : Erreur quadratique moyenne négative

# Le score sera négatif, donc prenez l'opposé pour obtenir l'erreur quadratique moyenne positive
rmse_scores = (-scores) ** 0.5

# Calculez la moyenne des scores RMSE
mean_rmse_score = rmse_scores.mean()
print("Moyenne des scores RMSE :", mean_rmse_score)

Moyenne des scores RMSE : 48.96389922649845


***Model 5***

In [70]:
from sklearn.svm import SVR

# Créez un modèle SVR
model = SVR(kernel='linear', C=1.0)

# Entraînez le modèle
model.fit(X, y)

SVR(kernel='linear')

In [71]:
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')  # Métrique de performance : Erreur quadratique moyenne négative

# Le score sera négatif, donc prenez l'opposé pour obtenir l'erreur quadratique moyenne positive
rmse_scores = (-scores) ** 0.5

# Calculez la moyenne des scores RMSE
mean_rmse_score = rmse_scores.mean()
print("Moyenne des scores RMSE :", mean_rmse_score)

KeyboardInterrupt: 

In [ ]:
import xgboost as xgb

# Créez un modèle XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)

# Entraînez le modèle
model.fit(X, y)

scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')  # Métrique de performance : Erreur quadratique moyenne négative

# Le score sera négatif, donc prenez l'opposé pour obtenir l'erreur quadratique moyenne positive
rmse_scores = (-scores) ** 0.5

# Calculez la moyenne des scores RMSE
mean_rmse_score = rmse_scores.mean()
print("Moyenne des scores RMSE :", mean_rmse_score)


In [36]:
mae

48.9046834393692